In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torchvision.models as models



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os     
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.init
from keras.preprocessing.image import load_img
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
root = '/kaggle/input/lottea/shape'

In [ ]:
#constrains
WIDTH = 256
HEIGHT = 256
IMAGE_SIZE=(WIDTH,HEIGHT)
CHANNELS = 3


device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


In [ ]:
ft = nn.Sequential(
            nn.Conv2d(3,64,(3,3), padding=1), # (256,256,64)
            nn.ReLU(True),
            nn.Conv2d(64,64,(3,3), padding=1), # (256,256,64)
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),#(128,128,64)
            
            nn.Conv2d(64,128,(3,3), padding=1),#(128,128,128)
            nn.ReLU(True),  
            nn.Conv2d(128,128,(3,3), padding=1),#(128,128,128)
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),#(64,64,128)
    
            nn.Conv2d(128,256,(3,3), padding=1),#(64,64,256)
            nn.ReLU(True),
            nn.Conv2d(256,256,(3,3), padding=1),#(64,64,256)
            nn.ReLU(True),
            nn.Conv2d(256,256,(3,3), padding=1),#(64,64,256)
            nn.ReLU(True),    
            nn.Conv2d(256,256,(3,3), padding=1),#(64,64,256)
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),#(32,32,256)
    
            nn.Conv2d(256,512,(3,3), padding=1),#(32,32,512)
            nn.ReLU(True),
            nn.Conv2d(512,512,(3,3), padding=1),#(32,32,512)
            nn.ReLU(True),
            nn.Conv2d(512,512,(3,3), padding=1),#(32,32,512)
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),#(16,16,512)
    
            nn.Conv2d(512,512,(3,3), padding=1),#(32,32,512)
            nn.ReLU(True),
            nn.Conv2d(512,512,(3,3), padding=1),#(32,32,512)
            nn.ReLU(True),
            nn.Conv2d(512,512,(3,3), padding=1),#(32,32,512)
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),#(8,8,512)
        
            
        )

class model(nn.Module):
    def __init__(self, features,num_classes=3):
        super(model, self).__init__()        
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((8, 8))
        
        self.classifier = nn.Sequential(
            nn.Linear(512*8*8,4096),
            nn.ReLU(True),
            nn.Linear(4096,4096),
            nn.ReLU(True),
            nn.Linear(4096,3),
        )#FC layer
        
        

    def forward(self, x):
        #print(x.shape)
        x = self.features(x) #Convolution 
        x = self.avgpool(x) # avgpool
        #print(x.shape)
        x = x.view(x.size(0), -1) #
        #print(x.shape)
        x = self.classifier(x) #FC layer
        
       
        return x

m = model(ft).to(device)
m

In [ ]:
vgg19 = models.vgg19()

In [40]:
vgg19.classifier = nn.Sequential(nn.Linear(512*8*8,4096),
            nn.ReLU(True),
            nn.Linear(4096,4096),
            nn.ReLU(True),
            nn.Linear(4096,3),)
vgg19.avgpool = nn.AdaptiveAvgPool2d((8, 8))
vgg19.to(device)
vgg19


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [41]:
# parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 50

# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(vgg19.parameters(), lr=learning_rate)

trans = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [42]:
from torchvision.datasets import ImageFolder 

train_dataset = ImageFolder(root = root,transform = trans)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)


In [43]:
total_batch = len(train_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in (train_loader):
        X = X.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = vgg19(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 406.012695
[Epoch:    2] cost = 0.965691864
[Epoch:    3] cost = 0.89280647
[Epoch:    4] cost = 0.852777481
[Epoch:    5] cost = 0.835857153
[Epoch:    6] cost = 0.814090371
[Epoch:    7] cost = 0.799780071
[Epoch:    8] cost = 0.789477408
[Epoch:    9] cost = 0.722921729
[Epoch:   10] cost = 0.693962514
Learning Finished!


In [46]:
torch.save(vgg19,'save10.pt')

In [ ]:
#vgg19=torch.load('/kaggle/working/save10.pt')

In [47]:
from torch.utils.data import DataLoader
from torchvision.io import read_image

In [48]:
trans2 = transforms.Compose([transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [49]:
import glob

path = "./*"
file_list = glob.glob('/kaggle/input/lottea/test/*')
file_list[:10]


['/kaggle/input/lottea/test/pqfzpfftvh.jpg',
 '/kaggle/input/lottea/test/aduzytppm.jpg',
 '/kaggle/input/lottea/test/llhvthqzl.jpg',
 '/kaggle/input/lottea/test/cetgibur.jpg',
 '/kaggle/input/lottea/test/gksljyjdu.jpg',
 '/kaggle/input/lottea/test/cruqoedwo.jpg',
 '/kaggle/input/lottea/test/frnkicftfp.jpg',
 '/kaggle/input/lottea/test/taivqvca.jpg',
 '/kaggle/input/lottea/test/tfzbhtce.jpg',
 '/kaggle/input/lottea/test/xzcaspjcue.jpg']

In [50]:

class DataSet:
    
    def __init__(self, root):
        self.images = []
        self.label = 0
        file_list = glob.glob('/kaggle/input/lottea/test/*')
        file_list.sort()
        for i in trange(len(file_list)):
            self.images.append(read_image(file_list[i]))
    def __len__(self):
    
        return len(self.images)
    def __getitem__(self, idx):
        """ Here we have to return the item requested by `idx`
            The PyTorch DataLoader class will use this method to make an iterable for
            our training or validation loop.
        """
        img = self.images[idx]
        img = img.unsqueeze(0)
        return img.float().to(device)
    def setlabel(self,l):
        self.label = l
 

In [ ]:
from tqdm import trange

In [51]:
ds = DataSet('/kaggle/input/lottea/test/')

100%|██████████| 19728/19728 [00:26<00:00, 741.00it/s]


In [52]:
prediction = vgg19(read_image('/kaggle/input/lottea/test/aadswotfh.jpg').unsqueeze(0).float().to(device))
print(prediction)
print(torch.argmax(prediction).item())
print(read_image('/kaggle/input/lottea/shape/box/123/10.jpg'))

tensor([[ 11.2831, -13.8048,  10.1879]], device='cuda:0',
       grad_fn=<AddmmBackward>)
0
tensor([[[184, 184, 184,  ..., 178, 178, 178],
         [184, 184, 184,  ..., 178, 178, 178],
         [184, 184, 184,  ..., 178, 178, 178],
         ...,
         [198, 198, 198,  ..., 197, 197, 197],
         [198, 198, 198,  ..., 197, 197, 197],
         [198, 198, 198,  ..., 197, 197, 197]],

        [[192, 192, 192,  ..., 186, 186, 186],
         [192, 192, 192,  ..., 186, 186, 186],
         [192, 192, 192,  ..., 186, 186, 186],
         ...,
         [214, 214, 214,  ..., 210, 210, 210],
         [214, 214, 214,  ..., 210, 210, 210],
         [214, 214, 214,  ..., 210, 210, 210]],

        [[195, 195, 195,  ..., 189, 189, 189],
         [195, 195, 195,  ..., 189, 189, 189],
         [195, 195, 195,  ..., 189, 189, 189],
         ...,
         [230, 230, 230,  ..., 226, 226, 226],
         [230, 230, 230,  ..., 226, 226, 226],
         [230, 230, 230,  ..., 226, 226, 226]]], dtype=torch.ui

In [55]:
result = []

with torch.no_grad():
    for i in trange (ds.__len__()):
    
        prediction = vgg19(ds.__getitem__(i)).squeeze()
        result.append(torch.argmax(prediction).item())
        ##print(prediction)

    prediction_np = np.array(result)
    print(result)
    file=pd.read_csv('/kaggle/input/lottea/sample_submission.csv')
    file['predictions'] =result
    file.to_csv('result.csv', index=False)

100%|██████████| 19728/19728 [04:37<00:00, 71.19it/s]

[0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 2, 1, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 1, 2, 0, 0, 0, 2, 0, 1, 1, 0, 1, 1, 0, 2, 0, 0, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 0, 1, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 1, 2, 1, 1, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 1, 2, 0, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 2, 0, 0, 2, 2, 0, 2, 1, 0, 0, 1, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 2, 1, 2, 1, 0, 0, 0, 0, 2, 1, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 

In [ ]:
!pip install tqdm